<a href="https://colab.research.google.com/github/Chen-Jialin886/-/blob/main/medical_code2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 实战：使用 unsloth 微调算命大师模型

In [ ]:
!nvidia-smi

Mon Mar 17 02:21:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pwd

/content


In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/colab

/content/drive/MyDrive/colab


##  第一步：安装依赖

In [5]:
# 第一步：安装依赖

# 安装 unsloth 包。unsloth 是一个用于微调大型语言模型（LLM）的工具，可以让模型运行更快、占用更少内存。
!pip install unsloth

# 卸载当前已安装的 unsloth 包（如果已安装），然后从 GitHub 的源代码安装最新版本。
# 这样可以确保我们使用的是最新功能和修复。
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# 安装 bitsandbytes 和 unsloth_zoo 包。
# bitsandbytes 是一个用于量化和优化模型的库，可以帮助减少模型占用的内存。
# unsloth_zoo 可能包含了一些预训练模型或其他工具，方便我们使用。
!pip install bitsandbytes unsloth_zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Found existing installation: unsloth 2025.3.14
Uninstalling unsloth-2025.3.14:
  Successfully uninstalled unsloth-2025.3.14
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-npuoesvy
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-npuoesvy
  Resolved https://github.com/unslothai/unsloth.git to commit 6f7c8c6d0a63caaa129cc0bc6b845d5d8b9c81e8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.14-py3-none-any.whl size=194396 sha256=b177d524805c8a4070053e7e8d5e8d941b7f23efc18a22ef9a3ffd3ecebb5c48
  Stored in directory: /tmp/pip-ephem-wheel-cache-m2xfjn8r/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth


## 第二步：加载训练模型

可根据需求替换模型：model_name="unsloth/DeepSeek-R1-Distill-Llama-8B"
基于 Llama 的 DeepSeek-R1 蒸馏版本，80 亿参数），如果大家想更换成其他的模型，直接改这个参数即可，比如可以改成：`unsloth/DeepSeek-R1-Distill-Qwen-7`
- 4bit 量化（4bit Quantization）：一种技术，通过减少模型的精度来节省内存，就像把一个大箱子压缩成一个小箱子，方便携带。

In [6]:
# 导入FastLanguageModel类，用来加载和使用模型

from unsloth import FastLanguageModel
import torch  # 导入torch工具，用于处理模型的数学运算

# 设置模型参数
max_seq_length = 2048  # 设置模型处理文本的最大长度，相当于给模型设置一个“最大容量”
dtype = None  # 设置数据类型，让模型自动选择最适合的精度
load_in_4bit = True  # 使用4位量化来节省内存，就像把大箱子压缩成小箱子

# 加载预训练模型，并获取tokenizer工具
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # 指定要加载的模型名称
    max_seq_length=max_seq_length,  # 使用前面设置的最大长度
    dtype=dtype,  # 使用前面设置的数据类型
    load_in_4bit=load_in_4bit  # 使用4位量化
    # token="hf_...",  # 如果需要访问授权模型，可以在这里填入密钥
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothBCOTrainer: No module named 'UnslothBCOTrainer'. Using tempfile instead!
==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## 第三步：微调前测试

In [8]:


# 定义提示风格的字符串模板，用于格式化问题
train_prompt_style = """
以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令:
你是一位经验丰富的医疗专家。
请回答以下健康咨询问题。

### 问题:
{}

### 回答:
<think>{}</think>
"""

# 定义具体的医疗问题
question = "1992年出生的女性，最近经常感到疲劳，想了解可能的健康问题及建议。"


微调前的输出结果

In [9]:
FastLanguageModel.for_inference(model)
# 准备模型以进行推理

inputs = tokenizer([train_prompt_style.format(question, "")], return_tensors="pt").to("cuda")
# 使用 tokenizer 对格式化后的问题进行编码，并移动到 GPU

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
# 使用模型生成回答

response = tokenizer.batch_decode(outputs)
# 解码模型生成的输出为可读文本

print(response[0])
# 打印生成的回答部分

<｜begin▁of▁sentence｜>
以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令:
你是一位经验丰富的医疗专家。
请回答以下健康咨询问题。

### 问题:
1992年出生的女性，最近经常感到疲劳，想了解可能的健康问题及建议。

### 回答:
<think></think>
嗯，好的，我现在要帮一位1992年出生的女性解决最近经常感到疲劳的问题。首先，我需要了解她的具体情况，包括她最近的生活习惯、饮食、工作和压力水平。因为疲劳可能有很多原因，比如睡眠不足、营养不良、压力大或者缺乏运动等等。

我应该建议她先调整作息，保证充足的睡眠时间。然后，饮食方面要多吃富含蛋白质、维生素和矿物质的食物，比如瘦肉、鱼、全谷物和蔬菜。适量运动也很重要，可以推荐一些容易开始的运动，比如散步、瑜伽或者家里锻炼。

同时，我要提醒她注意身体的信号，适当休息，避免过度劳累。如果疲劳情况严重，可能需要检查血液相关的疾病，比如贫血、甲状腺问题或者感染。心理压力大也是一个因素，可能需要寻求心理咨询的帮助。

最后，建议她定期体检，了解自己的身体状况，及时发现和解决问题。保持积极的心态和良好的生活习惯，才能更好地应对疲劳问题。
</think>

作为1992年出生的女性，您可能正经历一些身体和心理上的变化。以下是一些可能导致您感到疲劳的原因及建议：

1. **睡眠不足**：确保每天有足够的睡眠时间（7-9小时），避免熬夜或长时间工作。

2. **饮食不均衡**：多摄入富含蛋白质、维生素和矿物质的食物，如瘦肉、鱼、全谷物和蔬菜，少吃高糖、高油和加工食品。

3. **缺乏运动**：每周进行至少150分钟的中等强度运动（如快走、瑜伽或游泳）或75分钟的高强度运动（如跑步或骑自行车）。

4. **压力管理**：如果您感到压力大，可能需要学习一些放松技巧，如深呼吸、冥想或瑜伽，或者寻求心理咨询的帮助。

5. **贫血或其他血液问题**：如果疲劳持续超过一周，建议就医检查血液指标，排除贫血、甲状腺问题或感染的可能性。

6. **感染**：常见的感染，如细菌感染、病毒感染或寄生虫感染，也可能导致疲劳，及时就医检查是必要的。

7. **激素变化**：女性的激

## 第四步：加载数据集

In [10]:
# 定义结束标记 (EOS_TOKEN)，用于指示文本的结束
EOS_TOKEN = tokenizer.eos_token  # 必须添加结束标记

# 导入数据集加载函数
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
#ds = load_dataset("FreedomIntelligence/Medical-R1-Distill-Data-Chinese")

# 加载指定的数据集，选择中文语言和训练集的前200条记录
dataset = load_dataset("FreedomIntelligence/Medical-R1-Distill-Data-Chinese",'zh', split="train[0:200]", trust_remote_code=True)

# 打印数据集的列名，查看数据集中有哪些字段
print(dataset.column_names)

README.md:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

medical_r1_distill_sft_Chinese.json:   0%|          | 0.00/124M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17000 [00:00<?, ? examples/s]

['question', 'reasoning (reasoning_content)', 'response (content)']


需要注意的是，这里字段格式和前面提到的格式略有区别，除了包含基本的问题（question）、回答（responese），还包含了模型的思考过程（reasoning），因为我们现在要训练的是一个推理模型，所以数据集中最好也要包含模型的思考过程，这样训练出来的推理模型效果更好。




对数据集进行格式化，再打印出来一条看看：

In [11]:
def formatting_prompts_func(examples):
    # 从数据集中提取问题、复杂思考过程和回答
    inputs = examples["question"]
    cot = examples["reasoning (reasoning_content)"]
    outputs = examples["response (content)"]
    texts = []  # 用于存储格式化后的文本
    # 遍历每个问题、思考过程和回答，进行格式化
    for input, cot, output in zip(inputs, cot, outputs):
        # 使用字符串模板插入数据，并加上结束标记
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)  # 将格式化后的文本添加到列表中
    return {
        "text": texts,  # 返回包含所有格式化文本的字典
    }

dataset = dataset.map(formatting_prompts_func, batched=True)
dataset["text"]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

['\n以下是描述任务的指令，以及提供进一步上下文的输入。\n请写出一个适当完成请求的回答。\n在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。\n\n### 指令:\n你是一位经验丰富的医疗专家。\n请回答以下健康咨询问题。\n\n### 问题:\n根据患者59岁女性，出现胸闷、憋喘症状和相关的体格检查结果，其中X线显示双侧胸腔积液，胸水检查结果显示血性积液且有异常细胞，痰细胞学检查提示癌细胞体积小、圆形或卵圆形、胞质少，核畸形并深染，结合这些检查结果，患者最可能的诊断是什么？\n\n### 回答:\n<think>嗯，好的，我现在要解决这个病例的问题。首先，患者是59岁女性，主诉是胸闷和憋喘。体格检查发现有双侧胸腔积液，X线确认了这一点。胸水检查显示是血性积液，并且有异常细胞。痰细胞学检查发现癌细胞体积小，呈圆形或卵圆形，胞质少，核畸形且深染。我需要根据这些信息来推断最可能的诊断。\n\n首先，我应该回顾一下胸腔积液的常见原因。胸腔积液可能由多种原因引起，包括充血性心力衰竭、感染（如肺炎旁积液）、恶性肿瘤、肺栓塞、结核等。但这里胸水是血性的，并且发现了异常细胞，所以恶性肿瘤的可能性更大。血性胸腔积液常见于恶性肿瘤，尤其是肺癌转移至胸膜，或者间皮瘤，或者其他转移癌，比如乳腺癌、卵巢癌等转移到胸膜。但患者是女性，59岁，需要考虑原发肺癌的可能。\n\n接下来，痰细胞学检查的结果提示癌细胞体积小、圆形或卵圆形，胞质少，核畸形深染。这些特征可能指向某种特定类型的肺癌。肺癌的主要类型有小细胞癌、鳞癌、腺癌和大细胞癌。其中，小细胞肺癌的细胞学特征通常是细胞小，呈圆形或卵圆形，胞质少，核染色深，核畸形可能不太明显，但可能有挤压现象。而腺癌的细胞通常较大，胞质可能较多，形成腺泡状结构，核可能较大，但这里的描述是体积小，所以可能更倾向于小细胞癌。\n\n不过，小细胞肺癌通常发生在中央型，容易早期转移，常见于吸烟者。患者是否有吸烟史呢？但题目中没有提到，所以不能确定。但痰中发现癌细胞，说明肿瘤可能位于支气管内，容易脱落到痰中，这可能更符合中央型肺癌，如小细胞癌或鳞癌。但鳞癌的细胞学特征通常是较大的细胞，胞质较多，角化现象，核深染但可能有更明显的核仁，而这里的描述是细胞小，所以可能更符合小细胞癌。\n\n另外，双侧胸腔积液，胸水中有异常细胞，可能提示胸膜

In [12]:
FastLanguageModel.for_training(model)

# 假设model是一个已经加载好的预训练模型
model = FastLanguageModel.get_peft_model(
    model,  # 传入已经加载好的预训练模型
    r = 16,  # 设置LoRA的秩，决定添加的可训练参数数量
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",  # 指定模型中需要微调的关键模块
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # 设置LoRA的超参数，影响可训练参数的训练方式
    lora_dropout = 0,  # 设置防止过拟合的参数，这里设置为 0 表示不丢弃任何参数
    bias = "none",  # 设置是否添加偏置项，这里设置为 "none" 表示不添加
    use_gradient_checkpointing = "unsloth",  # 使用优化技术节省显存并支持更大的批量大小
    random_state = 3407,  # 设置随机种子，确保每次运行代码时模型的初始化方式相同
    use_rslora = False,  # 设置是否使用 Rank Stabilized LoRA 技术，这里设置为 False 表示不使用
    loftq_config = None,  # 设置是否使用 LoftQ 技术，这里设置为 None 表示不使用
)

# 执行微调
model.for_training()

Unsloth 2025.3.14 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

配置微调参数：
*   学习率（Learning Rate）：通过 `TrainingArguments` 中的 `learning_rate` 参数设置的，这里的值为 `2e-4`（即 0.0002）。

*   批量大小（Batch Size）：由两个参数共同决定（实际的批量大小：`per_device_train_batch_size * gradient_accumulation_steps`，也就是 `2 * 4 = 8`）：

*   `per_device_train_batch_size`：每个设备（如 GPU）上的批量大小。

*   `gradient_accumulation_steps`：梯度累积步数，用于模拟更大的批量大小。

*   训练轮数（Epochs）：通过 `max_steps`(最大训练步数) 和数据集大小计算得出，在这段代码中，最大训练 70 步，每一步训练 8 个，数据集大小为 200，那训练论数就是 `70 * 8 / 200 = 3`

In [13]:
from trl import SFTTrainer  # 导入 SFTTrainer，用于监督式微调
from transformers import TrainingArguments  # 导入 TrainingArguments，用于设置训练参数
from unsloth import is_bfloat16_supported  # 导入函数，检查是否支持 bfloat16 数据格式

trainer = SFTTrainer(  # 创建一个 SFTTrainer 实例
    model=model,  # 传入要微调的模型
    tokenizer=tokenizer,  # 传入 tokenizer，用于处理文本数据
    train_dataset=dataset,  # 传入训练数据集
    dataset_text_field="text",  # 指定数据集中文本字段的名称
    max_seq_length=max_seq_length,  # 设置最大序列长度
    dataset_num_proc=2,  # 设置数据处理的并行进程数
    packing=False,  # 是否启用打包功能（这里设置为 False，打包可以让训练更快，但可能影响效果）
    args=TrainingArguments(  # 定义训练参数
        per_device_train_batch_size=2,  # 每个设备（如 GPU）上的批量大小
        gradient_accumulation_steps=4,  # 梯度累积步数，用于模拟大批次训练
        warmup_steps=5,  # 预热步数，训练开始时学习率逐渐增加的步数
        max_steps=75,  # 最大训练步数
        learning_rate=2e-4,  # 学习率，模型学习新知识的速度
        fp16=not is_bfloat16_supported(),  # 是否使用 fp16 格式加速训练（如果环境不支持 bfloat16）
        bf16=is_bfloat16_supported(),  # 是否使用 bfloat16 格式加速训练（如果环境支持）
        logging_steps=1,  # 每隔多少步记录一次训练日志
        optim="adamw_8bit",  # 使用的优化器，用于调整模型参数
        weight_decay=0.01,  # 权重衰减，防止模型过拟合
        lr_scheduler_type="linear",  # 学习率调度器类型，控制学习率的变化方式
        seed=3407,  # 随机种子，确保训练结果可复现
        output_dir="outputs",  # 训练结果保存的目录
        report_to="none",  # 是否将训练结果报告到外部工具（如 WandB），这里设置为不报告
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

## 第六步：执行训练

In [14]:
trainer_stats=trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 3 | Total steps = 75
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.636000
2,1.672800
3,1.554000
4,1.578800
5,1.558800
6,1.541000
7,1.538800
8,1.468300
9,1.369100
10,1.317800


## 第七步：微调后测试

In [17]:
# 打印前面的问题
print(question)  # 打印前面的问题

# 将模型切换到推理模式，准备回答问题
FastLanguageModel.for_inference(model)

# 将问题转换成模型能理解的格式，并发送到 GPU 上
inputs = tokenizer([train_prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 让模型根据问题生成回答，最多生成 1200 个新词
outputs = model.generate(
    input_ids=inputs.input_ids,  # 输入的数字序列
    attention_mask=inputs.attention_mask,  # 注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=1200,  # 最多生成 1200 个新词
    use_cache=True,  # 使用缓存加速生成
)

# 将生成的回答从数字转换回文字
response = tokenizer.batch_decode(outputs)

# 打印回答
print(response[0])

1992年出生的女性，最近经常感到疲劳，想了解可能的健康问题及建议。
<｜begin▁of▁sentence｜>
以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令:
你是一位经验丰富的医疗专家。
请回答以下健康咨询问题。

### 问题:
1992年出生的女性，最近经常感到疲劳，想了解可能的健康问题及建议。

### 回答:
<think></think>
好的，用户是一位1992年出生的女性，最近经常感到疲劳，想了解可能的健康问题及建议。首先，我需要分析她的年龄和症状，可能涉及哪些健康问题。她的年龄是30岁左右，处于生殖年龄，可能有怀孕、月经不调等情况，但她提到的是疲劳，所以可能需要排除其他原因。

首先，常见的原因可能包括缺乏睡眠、贫血、营养不良、感染、代谢疾病、心理压力、药物副作用、环境因素等。需要逐一考虑这些可能性。

缺乏睡眠是常见的原因，现代社会工作强度大，可能晚上熄灯后仍无法入睡，或者工作压力大，导致疲劳。贫血的话，可能需要检查血红蛋白水平，或者是否有贫血的症状如头晕、乏力等。营养不良可能涉及饮食习惯，如果她饮食不规律，缺乏蛋白质、维生素和矿物质，可能导致疲劳。感染方面，比如细菌性感染、病毒性感染或其他感染，可能需要体检和实验室检查。

代谢疾病方面，肥胖、糖尿病、高血压等都可能导致疲劳，需要评估体重、血糖、血脂水平。心理压力也是一个重要因素，长期的压力可能导致身体和心理疲劳，建议进行心理咨询或放松训练。

药物副作用，如果她正在服用某些药物，可能导致疲劳，比如抗抑郁药、抗凝血药等，需要查看用药记录。环境因素如空气污染、噪音、温度等，也可能影响睡眠和整体状态。

接下来，建议包括充足睡眠、健康饮食、适度运动、心理支持、定期体检、避免过度疲劳等。可能需要进一步的体检或实验室检查来确认具体原因，比如血常规、血糖、血脂、肝肾功能等。

另外，考虑到她的年龄，可能还需要关注生殖健康，比如怀孕或月经不调，但用户主要是关于疲劳，所以需要综合考虑。不过，如果疲劳持续，可能需要排除更严重的疾病，比如癌症、免疫缺陷等，但可能性较低，除非有其他症状如发热、体重减轻等。

总结起来，应该先进行全面的体检，做血液检查，评估生活习惯、饮食、心理状态等

# 实战：本地运行微调后的医疗问答模型

现在，我们已经在colab上完成完整的模型微调训练过程下一步开始使用微调后的模型。


## 讲微调后的模型保存为GGUF格式
将微调后的模型上传到 `Hugging Face Hub` 之前，我们先将它转换为 GGUF 格式。

`GGUF` 是一种高效的格式，它支持多种量化方法（如 4 位、8 位、16 位量化），可以显著减小模型文件的大小，便于存储和传输，适合在资源受限的设备上运行模型，例如在 Ollama 上部署时。量化后的模型在资源受限的设备上运行更快，适合边缘设备或低功耗场景。


In [5]:
# 导入 Google Colab 的 userdata 模块，用于访问用户数据
from google.colab import userdata

# 从 Google Colab 用户数据中获取 Hugging Face 的 API 令牌
HUGGINGFACE_TOKEN = userdata.get('token')

# 将模型保存为 8 位量化格式 (08_0)
# 这种格式文件小且运行快，适合部署到资源受限的设备
if True: model.save_pretrained_gguf("model", tokenizer,)

# 将模型保存为 16 位量化格式 (f16)
# 16 位量化精度更高，但文件稍大
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# 将模型保存为 4 位量化格式 (q4_k_m)
# 4 位量化文件最小，但精度可能稍低
if False: model.save_pretrained_gguf("model_q4", tokenizer, quantization_method = "q4_k_m")

SecretNotFoundError: Secret token does not exist.